# Machine Learning - Assignment 2

## Decision tree induction algorithm for classification tasks

The aim of the assignment is to:

* Implement a decision tree induction algorithm for classification tasks.
* Make sure it works for real valued features and nominal features (categorical features without rank, e.g., red - blue - green).
* Test the algorithm on 3 datasets.

Follow the instructions and implement what is missing to complete the assignment. Some functions have been started to help you a little bit with the inputs or outputs of the function.

**Note:** You might need to go back and forth during your implementation of the code. The structure is set up to make implementation easier, but how you return values from the different functions might vary, and you might find yourself going back and change something to make it easier later on.

## Assignment preparations

We help you out with importing the libraries.

**IMPORTANT NOTE:** You may not import any more libraries than the ones already imported!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Decision tree model

The main objective is to implement the decision tree model. The implemented decision tree needs to be recursive model, that is, it should be implemented general enough to call itself in order to grow. "Growing" a tree refers to the same thing as "training" a model.

As said in the introduction, the structure is set up to help with implementation, but the nature of this model makes it a bit harder to implement function-by-function. You will most likely go back and forth between these first tasks.

### 1) Grow Tree

We will start with the main function of the decision tree, the "growing" function. 

This function should be called when creating the model, but also from inside itself. It is responible for creating all the nodes and leafs in the tree.

### 2) Growth stopping conditions (or stopping criterias)

The "grow_tree" function needs some way of stop growing, otherwise it will grow indefinitely. We will adress this issue here.

The trees stopping criterias needs to handle the following:

1) When a node has only datapoints of a single class.

2) Prevent the tree from growing to large, i.e., a max depth.

3) Prevent the tree nodes from becoming to small.

4) Prevent the tree from growing when the node is large (has a lot of datapoints) but it is very unbalanced. This is an extention to case 1.

Can you think of some other stopping criterias that is good to have? 

In [ ]:
class Leaf:
    def __init__(self, majority_class: int) -> None:
        self.majority_class = majority_class

class Node:
    def __init__(self, left: Leaf | Node, right: Leaf | Node, split_feature: int, split_value: float | str) -> None:
        self.left = left
        self.right = right
        self.split_feature = split_feature
        self.split_value = split_value
    
    def next(self, data_point) -> Node | Leaf:
        # split feature is categorical
        if isinstance(self.split_value, str):
            if data_point[self.split_feature] == self.split_value:
                return self.left
            else:
                return self.right
        
        # split feature is numerical
        else:
            if data_point[self.split_feature] < self.split_value:
                return self.left
            else:
                return self.right
        


In [ ]:
def is_pure(labels) -> bool:
    return np.unique_values(labels).size == 1

def reached_max_depth(depth, max_depth) -> bool:
    return depth >= max_depth

def reached_min_node_size(data) -> bool:
    return data.shape[0] < 5

# Add more stopping criterias if needed. Don't forget to use them when growing the tree!

### 3) Best feature for splitting nodes

When we are growing the tree, we need to decide how we are going to split a node into two new nodes. This is achived by looking at the features of the data in the node and calculate the best feature to split on.

Here you have a choice:

* Split using **Information Entropy**
* Split using **Gini Impurity**

Finish the function below using Information Entropy or Gini Impurity.

**Note:** Your code should be able to handle both real and categorical features!

In [ ]:
def calculate_gini(feature, comparison_value, is_categorical: bool, labels) -> float:
    classes = np.unique_values(labels).tolist()
    # mark left and right nodes
    direction = np.where(feature == comparison_value, "left", "right") if is_categorical else np.where(feature < comparison_value, "left", "right")
    # calculate class distributions for both directions
    left_class_distribution, right_class_distribution = [0 for _ in classes], [0 for _ in classes]
    for row_index, dir in enumerate(direction):
        class_index = classes.index(labels[row_index])
        if dir == "left":
            left_class_distribution[class_index] += 1
        elif dir == "right":
            right_class_distribution[class_index] += 1
    # calculate gini index for both sides
    gini_left, gini_right = 1, 1
    left_totals, right_totals = sum(left_class_distribution), sum(right_class_distribution)
    if left_totals > 0:
        for class_count in left_class_distribution:
                gini_left -= (class_count / left_totals) ** 2  
    if right_totals > 0:
        for class_count in right_class_distribution:
            gini_right -= (class_count / right_totals) ** 2
    # calculate total weighted gini index
    gini_index = left_totals / (left_totals + right_totals) * gini_left + right_totals / (left_totals + right_totals) * gini_right
    return gini_index

In [ ]:
def select_splitting_feature(features, labels) -> tuple[int, float | str]:
    # save the best feature
    best_feature, best_gini, comparison_value = -1, 1, 0
    
    # iterate over all features
    for feature_index in range(features.shape[1]):
        feature = features[:,feature_index]

        # feature is categorical
        if feature.dtype == object:
            # get all feature values
            feature_values = np.unique_values(feature)
            # calculate best splitting value by calculating gini index for every value
            for feature_value in feature_values:
                gini_index = calculate_gini(feature, feature_value, True, labels)
                # update best feature with the current feature value
                if gini_index < best_gini:
                    best_feature = feature_index
                    best_gini = gini_index
                    comparison_value = feature_value

        # feature is numerical
        else:
            mean = np.mean(feature)
            gini_index = calculate_gini(feature, mean, False, labels)
            # update best feature
            if gini_index < best_gini:
                best_feature = feature_index
                best_gini = gini_index
                comparison_value = mean
            
    return (best_feature, comparison_value)

### 4) Split data

When growing the tree, we need to split the data multiple times, and what we decide to split varies a lot. It is similar to splitting data into train and test sets (remember from assignment 1), but we split the data based on the best feature for growing a good tree.

**IMPORTANT NOTE:** To calculate binary splits for real-valued features, the following rule must be applied: an instance with a feature value lower than the mean feature value follows the left edge from the split node while all other instances follow the right edge from the split node.

In [97]:
def split_data(features, labels, split_feature: int, comparison_value: float | str) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    left_features: np.ndarray; left_labels: np.ndarray; right_features: np.ndarray; right_labels: np.ndarray
    feature = features[:,split_feature]

    # feature is categorical
    if isinstance(comparison_value, str):
        element_mask = (feature == comparison_value)
        left_features = features[element_mask]
        left_labels = labels[element_mask]
        right_features = features[~element_mask]
        right_labels = labels[~element_mask]

    # feature is numerical
    else:
        element_mask = (feature < comparison_value)
        left_features = features[element_mask]
        left_labels = labels[element_mask]
        right_features = features[~element_mask]
        right_labels = labels[~element_mask]

    return (left_features, left_labels, right_features, right_labels)

In [ ]:
def grow_tree(features, labels, max_depth, depth = 0) -> Node | Leaf:
    # stopping criteria reached
    if is_pure(labels) or reached_max_depth(depth, max_depth) or reached_min_node_size(features):
        # calculate probabilities for each class
        values, counts = np.unique_counts(labels)
        index = np.argmax(counts)
        return Leaf(int(values[index]))
    else:
        best_feature, comparison_value = select_splitting_feature(features, labels)
        left_features, left_labels, right_features, right_labels = split_data(features, labels, best_feature, comparison_value)
        left_node = grow_tree(left_features, left_labels, max_depth, depth + 1)
        right_node = grow_tree(right_features, right_labels, max_depth, depth + 1)
        node = Node(left_node, right_node, best_feature, comparison_value)

    return node

### 5) Predict with tree model

Finally, when we have grown our tree, we would like to use it for prediction. When using the tree for prediction, we traverse the tree for each datapoint untill we land in a leaf node.

In [ ]:
def predict_with_tree(node: Node | Leaf, data_point) -> int:
    if isinstance(node, Leaf):
        return node.majority_class

    elif isinstance(node, Node):
        return predict_with_tree(node.next(data_point), data_point)

In [ ]:
def predict_all(node: Node | Leaf, test_set) -> list[int]:
    predictions = []
    for data_point in test_set:
        prediction = predict_with_tree(node, data_point)
        predictions.append(prediction)
    return predictions

## Test decision tree model, compare with scikit learn, and plot dataset results

In the last part of the lab, you are going to test your tree code and compare it to scikit learn. The goal is not to be better than an established library, but to give you an indication about if you are on the right track.

You will need to plot the results from your model and the scikit learn model using matplotlib. We suggest a simple but informative bar-charts.

To make the comparison fair, you should train and test both your decision tree algorithm and the scikit learn at least 5 times, and shuffle the data each time before splitting the data into a train and test set.

The datasets are:

* Wine - (https://archive.ics.uci.edu/dataset/109/wine)
* Heart disease - (https://archive.ics.uci.edu/dataset/45/heart+disease)
* Car - (https://archive.ics.uci.edu/dataset/19/car+evaluation)

**IMPORTANT NOTE 1:** Take note of the feature types in the datasets, some features are numerical in value but are in fact categorical features. Be sure to handle these features correctly!

**IMPORTANT NOTE 2:** In this assignment it helps to add an additional header with information about the features and if they are nominal (n) or real (r) features.

In [ ]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

You may use the "**accuracy_score**" function from scikit learn (imported above) to compare the performance of your own and scikit learns models.

See below for an example use.

In [ ]:
y_true = [1,1,1,1,1] # Pretend labels
y_pred = [1,1,2,2,1] # Pretend prediction

accuracy_score(y_true, y_pred)

### 6) Dataset 1: Wine

In [107]:
data_wine = pd.read_csv("wine.csv", header=2).to_numpy()

# TODO: Set up the data and split it into train and test-sets.
wine_train, wine_test = train_test_split(data_wine, test_size=0.2)

# TODO: Train and test your implemented tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model
skl_classifier = DecisionTreeClassifier(criterion='gini', max_depth=5)
skl_fitted_classifier = skl_classifier.fit(X=wine_train[:,:-1], y=wine_train[:,-1])
skl_predictions = skl_fitted_classifier.predict(wine_test[:,:-1])
skl_accuracy = accuracy_score(skl_predictions, wine_test[:,-1])

custom_classifier = grow_tree(wine_train[:,:-1], wine_train[:,-1], max_depth=5)
custom_predictions = predict_all(custom_classifier, wine_test[:,:-1])
print(custom_predictions)
print(wine_test[:,-1])
custom_accuracy = accuracy_score(custom_predictions, wine_test[:,-1])

print("skl", skl_accuracy)
print("custom", custom_accuracy)

# TODO: Train and test scikit learns tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

[2, 3, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 3, 3, 2, 3, 3, 3, 3, 1, 1, 1, 3, 3, 1, 2, 1, 2]
[2. 3. 2. 2. 2. 1. 2. 3. 1. 1. 2. 2. 1. 2. 1. 2. 2. 1. 2. 2. 3. 3. 2. 3.
 3. 3. 3. 1. 1. 1. 2. 3. 1. 3. 1. 2.]
skl 0.9444444444444444
custom 0.9166666666666666


### 7) Dataset 2: Heart Disease

In [ ]:
data_heart = pd.read_csv("heart.csv", header=1).to_numpy()

# TODO: Set up the data and split it into train and test-sets.

# TODO: Train and test your implemented tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Train and test scikit learns tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

### 8) Dataset 3: Car

In [ ]:
data_car = pd.read_csv("car.csv", header=1).to_numpy()

# TODO: Set up the data and split it into train and test-sets.

# TODO: Train and test your implemented tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Train and test scikit learns tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

### 9) Training with normalized data on the wine-dataset

So far, we have trained our decision trees with "raw" data, i.e., we haven't done much preprocessing on the data.

Here we will do minor preprocessing on the data with the help of the scikit-learn library: https://scikit-learn.org/stable/modules/preprocessing.html

In [ ]:
from sklearn import preprocessing

# TODO: Use the wine dataset from above and scale its features (Not the labels!) between 0 and 1

# TODO: Run the code from the dataset and compare the preprocessed vs non-preprocessed data.
# NOTE: You can copy most of the workflow from the dataset code above to save you some time.
# NOTE: Use the same train/test split for your tree model for both the preprocessed vs non-preprocessed data.

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

# Questions for examination:

In addition to completing the assignment with all its tasks, you should also prepare to answer the following questions:

1) Why is growing the tree indefinitely such a bad idea? The performance would increase would it not?

2) Beside preventing the tree from growing to large, what is the purpose of 'stopping criterias'?

3) What is the difference between **Information Entropy** and **Gini Impurity**?

4) What are some pros about using decision trees?

5) Did preprocessing the data help with performance when using decision trees?

# Finished!

Was part of the setup incorrect? Did you spot any inconsistencies in the assignment? Could something improve?

If so, please write them and send via email and send it to:

* marcus.gullstrand@ju.se

Thank you!